# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Environment, ScriptRunConfig
from azureml.core.model import InferenceConfig
from azureml.core import Environment
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, LocalWebservice

import os

ws = Workspace.from_config()

In [2]:
cluster_name = "compute-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Existing Compute using {cluster_name}")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)

Existing Compute using compute-cluster


## Dataset

The [train.py](./train.py) script gets data by calling `get_dataset` function, which is similar to the code below

In [3]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)
dataset = Dataset.get_by_name(ws, name='salary')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.
I am using a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model because in some preliminary testing it yielded pretty good results for my dataset. The two hyperparameters I am focusing are C and max_iter. I choose C to try different levels of regularization and choose max_iter to try different variations to see which parameters would lead to the best accuracy. I used the BanditPolicy to not waste compute as it will cancel the run early as soon the Accuracy start to become worse after doing an initial 10 runs. I used Random Parameter sampling to try a variety of different combinations of hyperparameters. The primary metric I used was Accuracy as I wanted to choose the parameters with the best hyperparameters.

In [4]:
early_termination_policy = BanditPolicy(slack_factor=0.15, delay_evaluation=10)

param_sampling = RandomParameterSampling({
    "--C": choice(0.01, 0.1, 1.0, 10.0),
    "--max_iter": choice(50, 100, 150, 200, 250)
})

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies_hyperdrive.yml')

estimator = ScriptRunConfig(script="train.py",
                        source_directory=".",
                        environment=sklearn_env,
                        compute_target=compute_target)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                    run_config=estimator,
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_concurrent_runs=4,
                                    max_total_runs=20
                                    )

In [5]:
from azureml.train.hyperdrive import HyperDriveRun
hd_run: HyperDriveRun = experiment.submit(hyperdrive_run_config)
hd_run.wait_for_completion()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details


TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
import os
from azureml.core.model import Model
os.makedirs("./outputs", exist_ok=True)
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_file_names = best_run.get_file_names()
print('Metrics: ', best_run_metrics)
print("\n\nBest run file names :", best_run_file_names)
print('\n\nRun Id: {} Accuracy: {}'.format(best_run.id, best_run_metrics['Accuracy']))


Metrics:  {'Regularization Strength:': 0.1, 'Max iterations:': 250, 'Accuracy': 0.8570551205281745}


Best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'logs/azureml/dataprep/rslex.log.2022-05-08-23', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'user_logs/std_log.txt']


Run Id: HD_53f9a8b5-1560-470b-8885-5cff19be56e1_16 Accuracy: 0.8570551205281745


In [8]:
final_model: Model = best_run.register_model(model_name = 'hyperdrive_model',model_path = "./outputs/model.joblib")
final_model.download(".")

'model.joblib'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [40]:
inference_config = InferenceConfig(
    environment=sklearn_env,
    source_directory=".",
    entry_script="./score.py"
)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)
# deployment_config = LocalWebservice.deploy_configuration(port=6789)


service = Model.deploy(
    ws,
    "hdmodelservice-aci",
    [final_model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-05-09 02:22:35+00:00 Creating Container Registry if not exists.
2022-05-09 02:22:35+00:00 Registering the environment.
2022-05-09 02:22:36+00:00 Use the existing image.
2022-05-09 02:22:36+00:00 Generating deployment configuration.
2022-05-09 02:22:37+00:00 Submitting deployment to compute.
2022-05-09 02:22:39+00:00 Checking the status of deployment hdmodelservice-aci..
2022-05-09 02:25:51+00:00 Checking the status of inference endpoint hdmodelservice-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [45]:
import requests
import json
from score import process_input
import pandas as pd

uri = service.scoring_uri
primary, secondary = service.get_keys()
print(uri)
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {primary}"
# 31, Private,45781, Masters,14, Never-married, Prof-specialty, Not-in-family, White, Female,14084,0,50, United-States, >50K
# 38, Private,215646, HS-grad,9, Divorced, Handlers-cleaners, Not-in-family, White, Male,0,0,40, United-States, <=50K

data_under_50k = {
    "age": [38],
    "workclass": ["Private"],
    "fnlwgt": [215646],
    "education": ["HS-grad"],
    "education-num": [9],
    "marital-status": ["Divorced"],
    "occupation": ["Handlers-cleaners"],
    "sex": ["Male"],
    "race": ["White"],
    "relationship": ["Not-in-family"],
    "capital-gain": [0],
    "capital-loss": [0],
    "hours-per-week": [40],
    "native-country": ["United-States"]
}

data = json.dumps(data_under_50k)
response = requests.post(uri, data=data, headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print(response.status_code)

data_over_50k = {
    "age": [31],
    "workclass": ["Private"],
    "fnlwgt": [45781],
    "education": ["Masters"],
    "education-num": [14],
    "marital-status": ["Never-married"],
    "occupation": ["Prof-specialty"],
    "relationship": ["Not-in-family"],
    "race": ["White"],
    "sex": ["Female"],
    "capital-gain": [14084],
    "capital-loss": [0],
    "hours-per-week": [50],
    "native-country": ["United-States"]
}

data = json.dumps(data_over_50k)
response = requests.post(uri, data=data, headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print(response.status_code)

# print(service.get_logs())

http://0fa454d8-d6e2-4205-a3b9-d685b5b9f0f7.southcentralus.azurecontainer.io/score
{"result": [0]}
{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
print(service.get_logs())

2022-05-09T02:24:53,289713000+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-05-09T02:24:53,294453200+00:00 - iot-server/run 
2022-05-09T02:24:53,293571300+00:00 - rsyslog/run 
2022-05-09T02:24:53,328087100+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-05-09T02:24:53,729893300+00:00 - iot-server/finish 1 0
2022-05-09T02:24:53,731503500+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (66)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 89
SPARK_HOME not set. Skipping PySpark Initialization.
NumExpr defaulting to 1 threads.
Initializing logger
2022-05-09 02:24:57,008 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-05-09 02:24:57,009 | root | INFO | Starting up request id generator
2022-05-09 

In [ ]:
# Clean up
service.delete()
compute_target.delete()